# Fetching locations

In [64]:
SELECT name, nameID as locationID, id as _id
FROM [iot].[locations];

(1 row affected)

Total execution time: 00:00:00.017

name,locationID,_id
Churchill 1C,churchill-1c,1


# Fetching data

In [ ]:
-- per minute
DECLARE @location VARCHAR(50);
SET @location = 'churchill-1c';

SELECT TOP 1000 dateadd(mi, datediff(mi, 0, timestamp), 0) as timestamp, AVG(temperatureDHT) as temperature, AVG(pressureLPS) as pressure, AVG(humidityDHT) as humidity, AVG(eco2) as eco2, AVG(wifiDevices) as wifiDevices, COUNT(timestamp) as '_count'
FROM [iot].[messages] m
INNER JOIN [iot].[locations] l
ON m.locationID = l.id and l.nameID = @location
GROUP BY dateadd(mi, datediff(mi, 0, timestamp), 0)
ORDER BY timestamp DESC

In [ ]:
-- per hour
DECLARE @location VARCHAR(50);
SET @location = 'churchill-1c';

SELECT TOP 1000 dateadd(hour, datediff(hour, 0, timestamp), 0) as timestamp, AVG(temperatureDHT) as temperature, AVG(pressureLPS) as pressure, AVG(humidityDHT) as humidity, AVG(eco2) as eco2, AVG(wifiDevices) as wifiDevices, COUNT(timestamp) as '_count'
FROM [iot].[messages] m
INNER JOIN [iot].[locations] l
ON m.locationID = l.id and l.nameID = @location
GROUP BY dateadd(hour, datediff(hour, 0, timestamp), 0)
ORDER BY timestamp DESC

In [ ]:
-- per X minutes
DECLARE @location VARCHAR(50);
SET @location = 'churchill-1c';

DECLARE @perMinutes INT;
SET @perMinutes = 20;

SELECT TOP 1000 dateadd(mi, datediff(mi,0, timestamp) / @perMinutes * @perMinutes, 0) as timestamp, AVG(temperatureDHT) as temperature, AVG(pressureLPS) as pressure, AVG(humidityDHT) as humidity, AVG(eco2) as eco2, AVG(wifiDevices) as wifiDevices, COUNT(timestamp) as '_count'
FROM [iot].[messages] m
INNER JOIN [iot].[locations] l
ON m.locationID = l.id and l.nameID = @location
GROUP BY dateadd(mi, datediff(mi,0, timestamp) / @perMinutes * @perMinutes, 0)
ORDER BY timestamp DESC